In [1]:
from python.simulation import Simulation
import pandas as pd
import os

In [7]:
help(Simulation)

Help on class Simulation in module python.simulation:

class Simulation(python.wrapper.python_lbm.PythonClient)
 |  Method resolution order:
 |      Simulation
 |      python.wrapper.python_lbm.PythonClient
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, lbmFile)
 |      Load a simulation from lbm file
 |      
 |      Arguments:
 |          lbmFile {str} -- File system path to the lbm file
 |  
 |  get_average_names(self)
 |      List the names of the time averaged measurement areas
 |      
 |      Returns:
 |          list -- List of name strings
 |  
 |  get_averages(self, avg_type)
 |      Get the time averaged values measured during the simulation
 |      
 |      Arguments:
 |          avg_type {str} -- temperature, velocity or flow
 |      
 |      Returns:
 |          DataFrame -- A dataframe of measured values for all areas
 |  
 |  get_boundary_condition_names(self)
 |      Get a list of named boundary 

In [2]:
sim = Simulation(f'{os.getcwd()}/problems/ocp/project.lbm')

In [9]:
sim.get_boundary_condition_names()

['P02HDZ01',
 'P02HDZ02',
 'P02HDZ03',
 'P02HDZ04',
 'P02R01C01SRV01',
 'P02R01C01SRV02',
 'P02R01C01SRV03',
 'P02R01C02SRV01',
 'P02R01C02SRV02',
 'P02R01C02SRV03',
 'P02R01C03SRV01',
 'P02R01C03SRV02',
 'P02R01C03SRV03',
 'P02R01C04SRV01',
 'P02R01C04SRV02',
 'P02R01C04SRV03',
 'P02R01C05SRV01',
 'P02R01C05SRV02',
 'P02R01C05SRV03',
 'P02R01C06SRV01',
 'P02R01C06SRV02',
 'P02R01C06SRV03',
 'P02R01C07SRV01',
 'P02R01C07SRV02',
 'P02R01C07SRV03',
 'P02R01C08SRV01',
 'P02R01C08SRV02',
 'P02R01C08SRV03',
 'P02R01C09SRV01',
 'P02R01C09SRV02',
 'P02R01C09SRV03',
 'P02R01C10SRV01',
 'P02R01C10SRV02',
 'P02R01C10SRV03',
 'P02R02C01SRV01',
 'P02R02C01SRV02',
 'P02R02C01SRV03',
 'P02R02C02SRV01',
 'P02R02C02SRV02',
 'P02R02C02SRV03',
 'P02R02C03SRV01',
 'P02R02C03SRV02',
 'P02R02C03SRV03',
 'P02R02C04SRV01',
 'P02R02C04SRV02',
 'P02R02C04SRV03',
 'P02R02C05SRV01',
 'P02R02C05SRV02',
 'P02R02C05SRV03',
 'P02R02C06SRV01',
 'P02R02C06SRV02',
 'P02R02C06SRV03',
 'P02R02C07SRV01',
 'P02R02C07SRV02'

In [3]:
sim.get_boundary_conditions('P02R04C07SRV03')

,type,temperature,velocity,normal,rel_pos,tau1,tau2,lambda
714,VoxelType.INLET_RELATIVE,7.436508,"[-0.05, 0.0, 0.0]","[-1, 0, 0]","[25, 0, 0]",533.333313,483.383698,0.0
713,VoxelType.INLET_ZERO_GRADIENT,NaN,"[-0.05, 0.0, 0.0]","[1, 0, 0]","[0, 0, 0]",0.000000,0.000000,0.0


In [4]:
sim.get_boundary_conditions()

,type,temperature,velocity,normal,rel_pos,tau1,tau2,lambda
0,VoxelType.FLUID,NaN,"[nan, nan, nan]","[0, 0, 0]","[0, 0, 0]",0.000000,0.000000,0.0
1,VoxelType.WALL,NaN,"[nan, nan, nan]","[1, 0, 0]","[0, 0, 0]",0.000000,0.000000,0.0
2,VoxelType.WALL,NaN,"[nan, nan, nan]","[-1, 0, 0]","[0, 0, 0]",0.000000,0.000000,0.0
3,VoxelType.WALL,NaN,"[nan, nan, nan]","[0, 1, 0]","[0, 0, 0]",0.000000,0.000000,0.0
4,VoxelType.WALL,NaN,"[nan, nan, nan]","[1, 1, 0]","[0, 0, 0]",0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...
752,VoxelType.INLET_RELATIVE,7.436508,"[0.05, 0.0, 0.0]","[1, 0, 0]","[-25, 0, 0]",533.333313,483.383698,0.0
753,VoxelType.INLET_ZERO_GRADIENT,NaN,"[0.05, 0.0, 0.0]","[-1, 0, 0]","[0, 0, 0]",0.000000,0.000000,0.0
754,VoxelType.INLET_RELATIVE,7.436508,"[0.05, 0.0, 0.0]","[1, 0, 0]","[-25, 0, 0]",533.333313,483.383698,0.0
755,VoxelType.INLET_ZERO_GRADIENT,NaN,"[0.05, 0.0, 0.0]","[-1, 0, 0]","[0, 0, 0]",0.000000,0.000000,0.0


In [5]:
sim.set_time_averaging_period(5.0)
sim.get_time_step()

0.0018193715950474143

In [6]:
sim.get_time()

datetime.datetime(1970, 1, 1, 1, 0)

In [7]:
sim.run(60.0)

In [8]:
sim.get_time()

datetime.datetime(1970, 1, 1, 1, 1, 0, 1036)

In [9]:
sim.get_averages("temperature")

,time,P02HDZ01_out,P02HDZ01_in,P02HDZ03_out,P02HDZ03_in,P02HDZ04_out,P02HDZ04_in,P02HDZ02_out,P02HDZ02_in,sensors_racks_04_to_06_in_b,...,sensors_racks_07_to_09_in_t,sensors_racks_04_to_06_out_b,sensors_racks_04_to_06_out_m,sensors_racks_04_to_06_out_t,sensors_racks_10_to_12_out_b,sensors_racks_10_to_12_out_m,sensors_racks_10_to_12_out_t,sensors_racks_01_to_03_out_b,sensors_racks_01_to_03_out_m,sensors_racks_01_to_03_out_t
0,1970-01-01 01:00:00.001819,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1970-01-01 01:00:05.003270,18.600119,19.715706,18.613150,19.737394,18.625494,19.734215,18.592621,19.672663,19.851538,...,19.794409,22.241241,22.576193,23.598806,22.667721,21.905729,22.503714,20.651169,22.304642,22.848473
2,1970-01-01 01:00:10.004721,18.599762,20.876717,18.599737,21.016495,18.635166,20.987385,18.624197,20.732239,19.361727,...,20.554245,22.850016,22.829359,24.166185,23.093370,22.543633,23.237133,20.760899,22.714371,23.479122
3,1970-01-01 01:00:15.006172,18.613741,22.005077,18.617834,22.140316,18.649035,22.044973,18.641079,21.798552,19.605732,...,21.605059,23.544590,23.096306,24.266594,24.291962,22.555779,23.653109,21.323566,22.841127,23.769266
4,1970-01-01 01:00:20.007622,18.595705,22.507477,18.617409,22.480860,18.667824,22.574947,18.603575,22.341476,18.657326,...,21.751534,23.877197,23.050367,24.307337,24.861696,22.738426,23.666094,21.475782,22.876556,23.781296
5,1970-01-01 01:00:25.009073,18.621899,22.659061,18.637281,22.750668,18.637377,22.818972,18.629128,22.656654,18.612688,...,22.373449,23.557514,23.034273,24.393930,24.283512,22.795177,23.680593,21.682627,22.955301,23.436247
6,1970-01-01 01:00:30.010524,18.569389,22.888000,18.637384,22.938208,18.656292,23.049644,18.631664,22.859716,18.883627,...,22.330782,23.557732,23.221119,24.389170,24.323954,22.839935,23.461655,22.044085,23.025976,23.461573
7,1970-01-01 01:00:35.011975,18.609217,22.879028,18.620029,22.970604,18.645586,23.046774,18.648623,22.937817,20.409946,...,22.938612,23.614532,23.170298,24.540047,24.699261,22.783735,23.811043,21.986475,23.051090,23.848959
8,1970-01-01 01:00:40.013426,18.624273,22.902969,18.634993,23.047201,18.629541,23.126799,18.658525,23.010513,20.366047,...,23.048176,23.709759,23.245445,24.632381,25.441492,22.914015,23.875875,21.916388,23.082489,24.094683
9,1970-01-01 01:00:45.014877,18.654974,22.952621,18.641922,23.060751,18.652420,23.119316,18.657026,23.043655,18.893581,...,23.324387,23.823631,23.216457,24.670719,25.312830,22.844809,24.057999,22.135529,23.249620,23.796059


In [10]:
srvs = []
# 3 servers per chassi, 10 chassis per rack, 12 racks
for rack in range(1, 13):
    for chassi in range(1, 11):
        for srv in range(1, 4):
            pad = lambda i : str(i).zfill(2)
            srvs += [f'P02R{pad(rack)}C{pad(chassi)}SRV{pad(srv)}']
temps = [100 for _ in range(0, len(srvs))]
vol_flows = [0.002 for _ in range(0, len(srvs))]

sim.set_boundary_conditions(srvs, temps, vol_flows)

In [11]:
sim.get_boundary_conditions()

,type,temperature,velocity,normal,rel_pos,tau1,tau2,lambda
0,VoxelType.FLUID,NaN,"[nan, nan, nan]","[0, 0, 0]","[0, 0, 0]",0.000000,0.000000,0.0
1,VoxelType.WALL,NaN,"[nan, nan, nan]","[1, 0, 0]","[0, 0, 0]",0.000000,0.000000,0.0
2,VoxelType.WALL,NaN,"[nan, nan, nan]","[-1, 0, 0]","[0, 0, 0]",0.000000,0.000000,0.0
3,VoxelType.WALL,NaN,"[nan, nan, nan]","[0, 1, 0]","[0, 0, 0]",0.000000,0.000000,0.0
4,VoxelType.WALL,NaN,"[nan, nan, nan]","[1, 1, 0]","[0, 0, 0]",0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...
752,VoxelType.INLET_RELATIVE,100.0,"[0.0050350754, 0.0, 0.0]","[1, 0, 0]","[-25, 0, 0]",533.333313,483.383698,0.0
753,VoxelType.INLET_ZERO_GRADIENT,100.0,"[0.0062938435, -0.0, -0.0]","[-1, 0, 0]","[0, 0, 0]",0.000000,0.000000,0.0
754,VoxelType.INLET_RELATIVE,100.0,"[0.0062938435, 0.0, 0.0]","[1, 0, 0]","[-25, 0, 0]",533.333313,483.383698,0.0
755,VoxelType.INLET_ZERO_GRADIENT,100.0,"[0.0062938435, -0.0, -0.0]","[-1, 0, 0]","[0, 0, 0]",0.000000,0.000000,0.0


In [14]:
sim.run(60.0)

In [15]:
sim.get_averages("temperature")[['time', 'sensors_racks_10_to_12_out_m']]

,time,sensors_racks_10_to_12_out_m
0,1970-01-01 01:00:00.001819,0.000000
1,1970-01-01 01:00:05.003270,21.905729
2,1970-01-01 01:00:10.004721,22.543633
3,1970-01-01 01:00:15.006172,22.555779
4,1970-01-01 01:00:20.007622,22.738426
5,1970-01-01 01:00:25.009073,22.795177
6,1970-01-01 01:00:30.010524,22.839935
7,1970-01-01 01:00:35.011975,22.783735
8,1970-01-01 01:00:40.013426,22.914015
9,1970-01-01 01:00:45.014877,22.844809
